In [1]:
from sklearn.metrics import accuracy_score
import import_ipynb
from config import *
import pandas as pd
import empyrical
import numpy as np
from sklearn.metrics import roc_curve, precision_recall_curve, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import os
import seaborn as sns

importing Jupyter notebook from config.ipynb


In [2]:
def print_overall_accuracy(dictionary):
    print('Training set score: {:.4f}'.format(dictionary["model"].score(dictionary["X_train"], dictionary["y_train"])))
    print('Test set score: {:.4f}'.format(dictionary["model"].score(dictionary["X_test"], dictionary["y_test"])))
    

In [3]:
def evaluate_overall_accuracy(dictionary):
    for labeling_method in dictionary:
        print(labeling_method)
        print_overall_accuracy(dictionary[labeling_method])
        print("-"*30)

In [4]:
def split_training_results_in_currencies_old(dictionary):
    
    # Data structure as following:
    # First layer dictionary containing all the labeling methods as key with dictionary as value
        # Second layer dictionary containing all the currencies as key with dictionary as value
            # Third layer dictionary containing 2 datasets (train and test dataframe with predicted and actual labels each)
    
    currencies = {}
    valid_currencies2 = {y: x for x, y in valid_currencies.items()}
    
    for labeling_method in dictionary:
        
        currencies[labeling_method] = {}
        
        dictionary[labeling_method]["X_train"]["true label"] = dictionary[labeling_method]["y_train"].values
        dictionary[labeling_method]["X_train"]["predicted label"] = dictionary[labeling_method]["y_pred_train"].values
        train_df = dictionary[labeling_method]["X_train"]
        
        dictionary[labeling_method]["X_test"]["true label"] = dictionary[labeling_method]["y_test"].values
        dictionary[labeling_method]["X_test"]["predicted label"] = dictionary[labeling_method]["y_pred_test"].values
        test_df = dictionary[labeling_method]["X_test"]
        
        #detokenize the currencies to make it readable
        train_df = train_df.replace({"currency": valid_currencies2})
        test_df = test_df.replace({"currency": valid_currencies2})
        
        for currency in valid_currencies.keys():  
            
            currencies[labeling_method][currency] = {}
            
            train_currency_df = train_df.loc[train_df['currency'] == currency]
            test_currency_df = test_df.loc[test_df['currency'] == currency]
            
            currencies[labeling_method][currency]["train"] = train_currency_df
            currencies[labeling_method][currency]["test"] = test_currency_df
            
    return currencies
            

In [5]:
def evaluate_currency_accuracy(dictionary):
    for labeling_method in dictionary:
        print(labeling_method)
        for currency in dictionary[labeling_method]:
            print(currency)
            accuracy_train = accuracy_score(dictionary[labeling_method][currency]["train"]["predicted label"], dictionary[labeling_method][currency]["train"]["true label"])
            print("Training score: " + str(accuracy_train))
            accuracy_test = accuracy_score(dictionary[labeling_method][currency]["test"]["predicted label"], dictionary[labeling_method][currency]["test"]["true label"])
            print("Test score: " + str(accuracy_test))
        print("-"*30)

In [6]:
def calculate_strategy_returns_old(df, time_horizon):
    """
    Calculate the relative return of an investment strategy over a specified time horizon.
    
    Parameters:
    df (pd.DataFrame): The dataframe containing at least the 'Price' column, 'predicted_label' column, and datetime index.
    time_horizon (int): The time horizon in days to calculate the relative return.
    
    Returns:
    pd.DataFrame: The dataframe with an additional column 'Strategy_Return' that contains the calculated strategy returns.
    """
    # Ensure the dataframe is sorted by the index
    df = df.sort_index()

    # Calculate the future price by shifting the 'Price' column by the time horizon
    df['Future_Price'] = df['Price'].shift(-time_horizon)

    # Calculate the relative return
    df['Relative_Return'] = (df['Future_Price'] - df['Price']) / df['Price']

    # Apply the trading strategy labels
    df['Strategy_Return'] = df['Relative_Return'] * df['predicted label']

    # For no trade, the return should be 0
    df.loc[df['predicted label'] == 0, 'Strategy_Return'] = 0

    # Drop the temporary 'Future_Price' and 'Relative_Return' columns
    df.drop(columns=['Future_Price', 'Relative_Return'], inplace=True)

    return df

In [7]:
def calculate_overall_strategy_return_old(df):
    
    # Initialize capital
    initial_capital = 1.0
    capital = initial_capital
    
    # Calculate the compounded capital over time
    for trade in df['Strategy_Return']:
        if pd.notna(trade):
            capital *= (1 + trade)

    # Calculate overall return
    overall_return = capital - initial_capital
    
    return overall_return

In [8]:
def calculate_overall_strategy_return(df, return_column):
    # Initialize capital
    initial_capital = 1.0
    capital = initial_capital
    
    # Calculate the compounded capital over time
    for trade in df[return_column]:
        if pd.notna(trade):
            capital *= (1 + trade)

    # Calculate overall return
    overall_return = capital - initial_capital
    
    return overall_return

In [9]:
def calculate_cumulative_returns(df):
    """
    Berechnet die kumulierten Returns basierend auf relativen Returns.

    Args:
    df (pd.DataFrame): Ein DataFrame mit einer Spalte "Strategy_Return", die die relativen Returns enthält.

    Returns:
    pd.Series: Eine Serie mit den kumulierten Returns.
    """
    # Berechnung des kumulierten Returns für den gesamten Datensatz
    cumulative_return = (df['Strategy_Return'] + 1).prod() - 1
    
    return cumulative_return

In [10]:
def calculate_sharpe_ratio_old(df):
    # Annahme eines risikofreien Zinssatzes
    risk_free_rate = 0.01

    # Berechnung des durchschnittlichen Returns
    average_return = df['Strategy_Return'].mean()

    # Berechnung der Standardabweichung des Returns
    std_dev_return = df['Strategy_Return'].std()

    # Berechnung der Sharpe Ratio
    sharpe_ratio = (average_return - risk_free_rate) / std_dev_return
    
    return sharpe_ratio

In [11]:
def calculate_sharpe_ratio(df, return_column):
    # Assume a risk-free rate
    risk_free_rate = 0.01

    # Calculate the average return
    average_return = df[return_column].mean()

    # Calculate the standard deviation of returns
    std_dev_return = df[return_column].std()

    # Calculate the Sharpe Ratio
    sharpe_ratio = (average_return - risk_free_rate) / std_dev_return if std_dev_return != 0 else 0.0
    
    return sharpe_ratio

In [12]:
def calculate_max_drawdown_old(df):
    """
    Berechnet den Maximum Drawdown (MDD) basierend auf relativen Returns.

    Args:
    df (pd.DataFrame): Ein DataFrame mit einer Spalte "Strategy_Return", die die relativen Returns enthält.

    Returns:
    float: Der Maximum Drawdown (MDD) für den gesamten Datensatz.
    """
    # Berechnung des kumulierten Returns
    df['Cumulative Return'] = (1 + df['Strategy_Return']).cumprod()

    # Berechnung des bisherigen Maximums des kumulierten Returns
    df['Cumulative Max'] = df['Cumulative Return'].cummax()

    # Berechnung des Drawdowns
    df['Drawdown'] = df['Cumulative Return'] / df['Cumulative Max'] - 1

    # Berechnung des Maximum Drawdown
    max_drawdown = df['Drawdown'].min()

    return max_drawdown

In [13]:
def calculate_max_drawdown(df, return_column):
    """
    Berechnet den Maximum Drawdown (MDD) basierend auf relativen Returns.

    Args:
    df (pd.DataFrame): Ein DataFrame mit einer Spalte für die relativen Returns.
    return_column (str): Der Name der Spalte, die die relativen Returns enthält.

    Returns:
    float: Der Maximum Drawdown (MDD) für den gesamten Datensatz.
    """
    # Berechnung des kumulierten Returns
    df['Cumulative Return'] = (1 + df[return_column]).cumprod()

    # Berechnung des bisherigen Maximums des kumulierten Returns
    df['Cumulative Max'] = df['Cumulative Return'].cummax()

    # Berechnung des Drawdowns
    df['Drawdown'] = df['Cumulative Return'] / df['Cumulative Max'] - 1

    # Berechnung des Maximum Drawdown
    max_drawdown = df['Drawdown'].min()

    # Drop the temporary columns
    df.drop(columns=['Cumulative Return', 'Cumulative Max', 'Drawdown'], inplace=True)

    return max_drawdown


In [14]:
def calculate_mean_return_old(df):
    """
    Berechnet den durchschnittlichen Return basierend auf relativen Returns.

    Args:
    df (pd.DataFrame): Ein DataFrame mit einer Spalte "Strategy_Return", die die relativen Returns enthält.

    Returns:
    float: Der durchschnittliche Return für den gesamten Datensatz.
    """
    
    # Berechnung des durchschnittlichen Returns
    mean_return = df['Strategy_Return'].mean()
    
    return mean_return

In [15]:
def calculate_mean_return(df, return_column):
    """
    Berechnet den durchschnittlichen Return basierend auf relativen Returns.

    Args:
    df (pd.DataFrame): Ein DataFrame mit einer Spalte für die relativen Returns.
    return_column (str): Der Name der Spalte, die die relativen Returns enthält.

    Returns:
    float: Der durchschnittliche Return für den gesamten Datensatz.
    """
    
    # Berechnung des durchschnittlichen Returns
    mean_return = df[return_column].mean()
    
    return mean_return


In [16]:
def calculate_hit_ratio_old(df):
    """
    Berechnet die Hit Ratio basierend auf relativen Returns.

    Args:
    df (pd.DataFrame): Ein DataFrame mit einer Spalte "Strategy_Return", die die relativen Returns enthält.

    Returns:
    float: Die Hit Ratio für den gesamten Datensatz.
    """
    
    # Berechnung der Hit Ratio
    total_trades = len(df)
    positive_trades = (df['Strategy_Return'] > 0).sum()
    hit_ratio = positive_trades / total_trades
    
    return hit_ratio

In [17]:
def calculate_hit_ratio(df, return_column):
    """
    Berechnet die Hit Ratio basierend auf relativen Returns.

    Args:
    df (pd.DataFrame): Ein DataFrame mit einer Spalte für die relativen Returns.
    return_column (str): Der Name der Spalte, die die relativen Returns enthält.

    Returns:
    float: Die Hit Ratio für den gesamten Datensatz.
    """
    
    # Berechnung der Hit Ratio
    total_trades = len(df)
    positive_trades = (df[return_column] > 0).sum()
    hit_ratio = positive_trades / total_trades if total_trades > 0 else 0.0
    
    return hit_ratio

In [18]:
def calculate_annual_return_old(df, periods):
    """
    Berechnet den annualisierten Return basierend auf relativen Returns.

    Args:
    df (pd.DataFrame): Ein DataFrame mit einer Spalte "Strategy_Return", die die relativen Returns enthält.
    periods_per_year (int): Die Anzahl der Perioden pro Jahr. Standardmäßig 252 für tägliche Returns.

    Returns:
    float: Der annualisierte Return für den gesamten Datensatz.
    """

    if periods == "daily":
        periods_per_year = 365
    if periods == "weekly":
        periods_per_year = 52
    
    mean_return_per_period = df['Strategy_Return'].mean()
    
    # Annualisierung des durchschnittlichen Returns
    annual_return = (1 + mean_return_per_period) ** periods_per_year - 1
    
    return annual_return

In [19]:
def calculate_annual_return(df, periods, return_column):
    """
    Berechnet den annualisierten Return basierend auf relativen Returns.

    Args:
    df (pd.DataFrame): Ein DataFrame mit einer Spalte für die relativen Returns.
    periods (str): Die Periode der Returns ("daily" oder "weekly").

    Returns:
    float: Der annualisierte Return für den gesamten Datensatz.
    """
    
    if periods == "daily":
        periods_per_year = 365
    elif periods == "weekly":
        periods_per_year = 52
    else:
        raise ValueError("Period must be 'daily' or 'weekly'")

    mean_return_per_period = df[return_column].mean()
    
    # Annualisierung des durchschnittlichen Returns
    annual_return = (1 + mean_return_per_period) ** periods_per_year - 1
    
    return annual_return


In [20]:
def calculate_annual_volatility_old(df, periods):
    """
    Berechnet die annualisierte Volatilität basierend auf relativen Returns.

    Args:
    df (pd.DataFrame): Ein DataFrame mit einer Spalte "Strategy_Return", die die relativen Returns enthält.
    periods_per_year (int): Die Anzahl der Perioden pro Jahr. Standardmäßig 252 für tägliche Returns.

    Returns:
    float: Die annualisierte Volatilität für den gesamten Datensatz.
    """
    
    if periods == "daily":
        periods_per_year = 365
    if periods == "weekly":
        periods_per_year = 52
        
    # Berechnung der Standardabweichung der Returns pro Periode
    std_dev_per_period = df['Strategy_Return'].std()
    
    # Annualisierung der Volatilität
    annual_volatility = std_dev_per_period * np.sqrt(periods_per_year)
    
    return annual_volatility


In [21]:
def calculate_annual_volatility(df, periods, return_column):
    """
    Berechnet die annualisierte Volatilität basierend auf relativen Returns.

    Args:
    df (pd.DataFrame): Ein DataFrame mit einer Spalte für die relativen Returns.
    periods (str): Die Periode der Returns ("daily" oder "weekly").
    return_column (str): Der Name der Spalte, die die relativen Returns enthält.

    Returns:
    float: Die annualisierte Volatilität für den gesamten Datensatz.
    """
    
    if periods == "daily":
        periods_per_year = 365
    elif periods == "weekly":
        periods_per_year = 52
    else:
        raise ValueError("Period must be 'daily' or 'weekly'")
        
    # Berechnung der Standardabweichung der Returns pro Periode
    std_dev_per_period = df[return_column].std()
    
    # Annualisierung der Volatilität
    annual_volatility = std_dev_per_period * np.sqrt(periods_per_year)
    
    return annual_volatility

In [22]:
def evaluate_strategy_financially_old(dictionary, periods):
    
    time_horizon = 1
    
    columns = ['labeling_method', 
               'currency', 
               'overall return',
               'sharpe ratio',
               'maximum drawdown',
               'mean return',
               'hit ratio',
               'annual return',
               'annual volatility']
    
    evaluation_df = pd.DataFrame(columns = columns)
    
    for labeling_method in dictionary:
        
        for currency in dictionary[labeling_method]:

            dictionary[labeling_method][currency]["test"] = calculate_strategy_returns(dictionary[labeling_method][currency]["test"], time_horizon)
            
            overall_return = calculate_overall_strategy_return(dictionary[labeling_method][currency]["test"])
            sharpe_ratio = calculate_sharpe_ratio(dictionary[labeling_method][currency]["test"])
            max_drawdown = calculate_max_drawdown(dictionary[labeling_method][currency]["test"])
            mean_return = calculate_mean_return(dictionary[labeling_method][currency]["test"])
            hit_ratio = calculate_hit_ratio(dictionary[labeling_method][currency]["test"])
            annual_return = calculate_annual_return(dictionary[labeling_method][currency]["test"], periods)
            annual_volatility = calculate_annual_volatility(dictionary[labeling_method][currency]["test"], periods)
            
            new_row = pd.DataFrame({'labeling_method': labeling_method, 
                                    'currency': currency, 
                                    'overall return': overall_return,
                                    'sharpe ratio': sharpe_ratio,
                                    'maximum drawdown': max_drawdown,
                                    'mean return': mean_return,
                                    'hit ratio': hit_ratio,
                                    'annual return': annual_return,
                                    'annual volatility': annual_volatility}, index=[0])
            evaluation_df = pd.concat([evaluation_df, new_row], ignore_index=True)
            
    return dictionary, evaluation_df

In [23]:
def calculate_strategy_returns_old(df, time_horizon, transaction_cost=0.006):
    """
    Calculate the relative return of an investment strategy over a specified time horizon with and without transaction costs.
    
    Parameters:
    df (pd.DataFrame): The dataframe containing at least the 'Price' column, 'predicted_label' column, and datetime index.
    time_horizon (int): The time horizon in days to calculate the relative return.
    transaction_cost (float): The transaction cost per trade as a fraction.
    
    Returns:
    pd.DataFrame: The dataframe with additional columns 'Strategy_Return' and 'Strategy_Return_TC' that contain the calculated strategy returns without and with transaction costs.
    """
    # Ensure the dataframe is sorted by the index
    df = df.sort_index()

    # Calculate the future price by shifting the 'Price' column by the time horizon
    df['Future_Price'] = df['Price'].shift(-time_horizon)

    # Calculate the relative return
    df['Relative_Return'] = (df['Future_Price'] - df['Price']) / df['Price']

    # Apply the trading strategy labels
    df['Strategy_Return'] = df['Relative_Return'] * df['predicted label']
    df['Strategy_Return_TC'] = df['Strategy_Return'] - transaction_cost * df['predicted label'].abs()

    # For no trade, the return should be 0
    df.loc[df['predicted label'] == 0, ['Strategy_Return', 'Strategy_Return_TC']] = 0

    # Drop the temporary 'Future_Price' and 'Relative_Return' columns
    df.drop(columns=['Future_Price', 'Relative_Return'], inplace=True)

    return df

In [24]:
def split_training_results_in_currencies(dictionary):
    
    currencies = {}
    valid_currencies2 = {y: x for x, y in valid_currencies.items()}
    
    for labeling_method in dictionary:
        
        currencies[labeling_method] = {}
        
        train_df = dictionary[labeling_method]["X_train"].copy()
        test_df = dictionary[labeling_method]["X_test"].copy()
        
        train_df["true label"] = dictionary[labeling_method]["y_train"].values
        train_df["predicted label"] = dictionary[labeling_method]["y_pred_train"].values
        
        test_df["true label"] = dictionary[labeling_method]["y_test"].values
        test_df["predicted label"] = dictionary[labeling_method]["y_pred_test"].values
        
        #detokenize the currencies to make it readable
        train_df = train_df.replace({"currency": valid_currencies2})
        test_df = test_df.replace({"currency": valid_currencies2})
        
        for currency in valid_currencies.keys():  
            
            currencies[labeling_method][currency] = {}
            
            train_currency_df = train_df.loc[train_df['currency'] == currency]
            test_currency_df = test_df.loc[test_df['currency'] == currency]
            
            currencies[labeling_method][currency]["train"] = train_currency_df
            currencies[labeling_method][currency]["test"] = test_currency_df
            
    return currencies

In [25]:
def evaluate_strategy_financially_old2(dictionary, periods, transaction_cost=0.006):
    time_horizon = 1
    
    columns = [
        'labeling_method', 
        'currency', 
        'overall_return', 'overall_return_TC',
        'sharpe_ratio', 'sharpe_ratio_TC',
        'maximum_drawdown', 'maximum_drawdown_TC',
        'mean_return', 'mean_return_TC',
        'hit_ratio', 'hit_ratio_TC',
        'annual_return', 'annual_return_TC',
        'annual_volatility', 'annual_volatility_TC'
    ]
    
    evaluation_df = pd.DataFrame(columns=columns)
    
    for labeling_method in dictionary:
        for currency in dictionary[labeling_method]:
            # Calculate strategy returns with and without transaction costs
            dictionary[labeling_method][currency]["test"] = calculate_strategy_returns(
                dictionary[labeling_method][currency]["test"], time_horizon, transaction_cost
            )
            
            # Calculate metrics without transaction costs
            overall_return = calculate_overall_strategy_return(dictionary[labeling_method][currency]["test"], 'Strategy_Return')
            sharpe_ratio = calculate_sharpe_ratio(dictionary[labeling_method][currency]["test"], 'Strategy_Return')
            max_drawdown = calculate_max_drawdown(dictionary[labeling_method][currency]["test"], 'Strategy_Return')
            mean_return = calculate_mean_return(dictionary[labeling_method][currency]["test"], 'Strategy_Return')
            hit_ratio = calculate_hit_ratio(dictionary[labeling_method][currency]["test"], 'Strategy_Return')
            annual_return = calculate_annual_return(dictionary[labeling_method][currency]["test"], periods, 'Strategy_Return')
            annual_volatility = calculate_annual_volatility(dictionary[labeling_method][currency]["test"], periods, 'Strategy_Return')
            
            # Calculate metrics with transaction costs
            overall_return_TC = calculate_overall_strategy_return(dictionary[labeling_method][currency]["test"], 'Strategy_Return_TC')
            sharpe_ratio_TC = calculate_sharpe_ratio(dictionary[labeling_method][currency]["test"], 'Strategy_Return_TC')
            max_drawdown_TC = calculate_max_drawdown(dictionary[labeling_method][currency]["test"], 'Strategy_Return_TC')
            mean_return_TC = calculate_mean_return(dictionary[labeling_method][currency]["test"], 'Strategy_Return_TC')
            hit_ratio_TC = calculate_hit_ratio(dictionary[labeling_method][currency]["test"], 'Strategy_Return_TC')
            annual_return_TC = calculate_annual_return(dictionary[labeling_method][currency]["test"], periods, 'Strategy_Return_TC')
            annual_volatility_TC = calculate_annual_volatility(dictionary[labeling_method][currency]["test"], periods, 'Strategy_Return_TC')
            
            # Append the results to the evaluation dataframe
            new_row = pd.DataFrame({
                'labeling_method': labeling_method, 
                'currency': currency, 
                'overall_return': overall_return,
                'overall_return_TC': overall_return_TC,
                'sharpe_ratio': sharpe_ratio,
                'sharpe_ratio_TC': sharpe_ratio_TC,
                'maximum_drawdown': max_drawdown,
                'maximum_drawdown_TC': max_drawdown_TC,
                'mean_return': mean_return,
                'mean_return_TC': mean_return_TC,
                'hit_ratio': hit_ratio,
                'hit_ratio_TC': hit_ratio_TC,
                'annual_return': annual_return,
                'annual_return_TC': annual_return_TC,
                'annual_volatility': annual_volatility,
                'annual_volatility_TC': annual_volatility_TC
            }, index=[0])
            
            evaluation_df = pd.concat([evaluation_df, new_row], ignore_index=True)
    
    return dictionary, evaluation_df

In [26]:
def evaluate_model_old(model_dict, labeling_method, save_path="visualizations/ml_metrics"):
    X_train = model_dict["X_train"]
    X_test = model_dict["X_test"]
    y_train = model_dict["y_train"]
    y_test = model_dict["y_test"]
    y_pred_test = model_dict["y_pred_test"]
    y_pred_train = model_dict["y_pred_train"]

    # Create directory if it doesn't exist
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    # Plot confusion matrix for test dataset
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    cm_test = confusion_matrix(y_test, y_pred_test)
    sns.heatmap(cm_test, annot=True, fmt="d", cmap="Blues", cbar=False)
    plt.title('Confusion Matrix - Test Data')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')

    # Plot confusion matrix for train dataset
    plt.subplot(1, 2, 2)
    cm_train = confusion_matrix(y_train, y_pred_train)
    sns.heatmap(cm_train, annot=True, fmt="d", cmap="Blues", cbar=False)
    plt.title('Confusion Matrix - Train Data')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')

    plt.tight_layout()

    # Save the plot
    save_name = os.path.join(save_path, f"confusion_matrix_{labeling_method}.png")
    plt.savefig(save_name)
    plt.close()

    # Print classification report for test dataset
    print("Classification Report - Test Data:")
    print(classification_report(y_test, y_pred_test))

    # Print classification report for train dataset
    print("\nClassification Report - Train Data:")
    print(classification_report(y_train, y_pred_train))


In [27]:
def plot_confusion_matrix(y_true, y_pred, classes, labeling_method, save_path="visualizations/ml_metrics"):
    # Create directory if it doesn't exist
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    cm = confusion_matrix(y_true, y_pred, labels=classes)

    plt.figure(figsize=(10, 10))
    sns.set(font_scale=1.2)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=classes, yticklabels=classes)

    plt.title('Confusion Matrix - Actuals and Predicted')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.xticks(rotation=45)

    # Save the plot
    save_name = os.path.join(save_path, f"confusion_matrix_{labeling_method}.png")
    plt.savefig(save_name)
    plt.close()

def evaluate_and_visualize_model(model_dict, labeling_method, save_path="visualizations/ml_metrics"):
    y_test = model_dict["y_test"]
    y_pred_test = model_dict["y_pred_test"]
    
    # Convert to numpy arrays
    y_test = y_test.to_numpy()
    y_pred_test = y_pred_test.to_numpy()
    
    # Ensure y_test and y_pred_test have same dimensions
    if y_test.ndim > 1:
        y_test = y_test.ravel()
    if y_pred_test.ndim > 1:
        y_pred_test = y_pred_test.ravel()
    
    classes = np.unique(np.concatenate((y_test, y_pred_test), axis=0))

    plot_confusion_matrix(y_test, y_pred_test, classes, labeling_method, save_path)

    # Print classification report for test dataset
    print("Classification Report - Test Data:")
    print(classification_report(y_test, y_pred_test))

In [28]:
def evaluate_and_visualize_all_approaches(dictionary):
    for labeling_method in dictionary:
        evaluate_and_visualize_model(dictionary[labeling_method], labeling_method)

In [29]:
def calculate_strategy_returns(df, transaction_cost=0.006):
    """
    Calculate the relative return of an investment strategy with and without transaction costs.
    
    Parameters:
    df (pd.DataFrame): The dataframe containing at least the 'Price' column, 'predicted_label' column, and datetime index.
    transaction_cost (float): The transaction cost per trade as a fraction.
    
    Returns:
    pd.DataFrame: The dataframe with additional columns 'Strategy_Return' and 'Strategy_Return_TC' that contain the calculated strategy returns without and with transaction costs.
    """
    # Ensure the dataframe is sorted by the index
    df = df.sort_index()

    # Calculate the percentage change in price
    df['Pct_Change'] = df['Price'].pct_change().fillna(0)

    # Calculate the strategy returns without transaction costs
    df['Strategy_Return'] = df['Pct_Change'] * df['predicted label']

    # Apply transaction costs only when there's a trade (label is -1 or 1)
    df['Transaction_Cost'] = df['predicted label'].abs() * transaction_cost

    # Calculate the strategy returns with transaction costs
    df['Strategy_Return_TC'] = df['Strategy_Return'] - df['Transaction_Cost']

    # For no trade, the return should be 0
    df.loc[df['predicted label'] == 0, ['Strategy_Return', 'Strategy_Return_TC']] = 0

    return df

def evaluate_strategy_financially(dictionary, periods, transaction_cost=0.006):
    """
    Evaluate the financial performance of various trading strategies and compare them to a Buy and Hold strategy.
    
    Parameters:
    dictionary (dict): Dictionary containing the trading strategies and their test data.
    periods (str): 'daily' or 'weekly' indicating the frequency of the data points.
    transaction_cost (float): The transaction cost per trade as a fraction.
    
    Returns:
    tuple: Updated dictionary with test results and a DataFrame containing the evaluation metrics.
    """
    columns = [
        'labeling_method', 
        'currency', 
        'overall_return', 'overall_return_TC',
        'sharpe_ratio', 'sharpe_ratio_TC',
        'maximum_drawdown', 'maximum_drawdown_TC',
        'mean_return', 'mean_return_TC',
        'hit_ratio', 'hit_ratio_TC',
        'annual_return', 'annual_return_TC',
        'annual_volatility', 'annual_volatility_TC'
    ]
    
    evaluation_df = pd.DataFrame(columns=columns)
    
    if periods not in ["daily", "weekly"]:
        raise ValueError("Invalid value for periods. Use 'daily' or 'weekly'.")

    for labeling_method in dictionary:
        for currency in dictionary[labeling_method]:
            # Calculate strategy returns with and without transaction costs
            strategy_df = dictionary[labeling_method][currency]["test"]
            strategy_df = calculate_strategy_returns(strategy_df, transaction_cost)
            dictionary[labeling_method][currency]["test"] = strategy_df
            
            # Metrics without transaction costs
            overall_return = calculate_overall_strategy_return(strategy_df, 'Strategy_Return')
            sharpe_ratio = calculate_sharpe_ratio(strategy_df, 'Strategy_Return')
            max_drawdown = calculate_max_drawdown(strategy_df, 'Strategy_Return')
            mean_return = calculate_mean_return(strategy_df, 'Strategy_Return')
            hit_ratio = calculate_hit_ratio(strategy_df, 'Strategy_Return')
            annual_return = calculate_annual_return(strategy_df, periods, 'Strategy_Return')
            annual_volatility = calculate_annual_volatility(strategy_df, periods, 'Strategy_Return')
            
            # Metrics with transaction costs
            overall_return_TC = calculate_overall_strategy_return(strategy_df, 'Strategy_Return_TC')
            sharpe_ratio_TC = calculate_sharpe_ratio(strategy_df, 'Strategy_Return_TC')
            max_drawdown_TC = calculate_max_drawdown(strategy_df, 'Strategy_Return_TC')
            mean_return_TC = calculate_mean_return(strategy_df, 'Strategy_Return_TC')
            hit_ratio_TC = calculate_hit_ratio(strategy_df, 'Strategy_Return_TC')
            annual_return_TC = calculate_annual_return(strategy_df, periods, 'Strategy_Return_TC')
            annual_volatility_TC = calculate_annual_volatility(strategy_df, periods, 'Strategy_Return_TC')
            
            new_row = pd.DataFrame({
                'labeling_method': labeling_method, 
                'currency': currency, 
                'overall_return': overall_return,
                'overall_return_TC': overall_return_TC,
                'sharpe_ratio': sharpe_ratio,
                'sharpe_ratio_TC': sharpe_ratio_TC,
                'maximum_drawdown': max_drawdown,
                'maximum_drawdown_TC': max_drawdown_TC,
                'mean_return': mean_return,
                'mean_return_TC': mean_return_TC,
                'hit_ratio': hit_ratio,
                'hit_ratio_TC': hit_ratio_TC,
                'annual_return': annual_return,
                'annual_return_TC': annual_return_TC,
                'annual_volatility': annual_volatility,
                'annual_volatility_TC': annual_volatility_TC
            }, index=[0])
            
            evaluation_df = pd.concat([evaluation_df, new_row], ignore_index=True)
    
    # Calculate metrics for Buy and Hold strategy for each currency, without considering transaction costs
    for currency in dictionary[labeling_method]:
        strategy_df = dictionary[labeling_method][currency]["test"]
        
        # Calculate buy-and-hold metrics based on the price percentage change
        returns_bh = strategy_df['Price'].pct_change().fillna(0)
        overall_return_BH = (returns_bh + 1).prod() - 1
        sharpe_ratio_BH = calculate_sharpe_ratio(strategy_df, 'Pct_Change')
        max_drawdown_BH = calculate_max_drawdown(strategy_df, 'Pct_Change')
        mean_return_BH = calculate_mean_return(strategy_df, 'Pct_Change')
        hit_ratio_BH = calculate_hit_ratio(strategy_df, 'Pct_Change')
        annual_return_BH = calculate_annual_return(strategy_df, periods, 'Pct_Change')
        annual_volatility_BH = calculate_annual_volatility(strategy_df, periods, 'Pct_Change')
        
        # Append the Buy and Hold results to the evaluation dataframe
        new_row_bh = pd.DataFrame({
            'labeling_method': 'Buy and Hold', 
            'currency': currency, 
            'overall_return': overall_return_BH,
            'overall_return_TC': overall_return_BH,  # No transaction costs considered for Buy and Hold
            'sharpe_ratio': sharpe_ratio_BH,
            'sharpe_ratio_TC': sharpe_ratio_BH,
            'maximum_drawdown': max_drawdown_BH,
            'maximum_drawdown_TC': max_drawdown_BH,
            'mean_return': mean_return_BH,
            'mean_return_TC': mean_return_BH,
            'hit_ratio': hit_ratio_BH,
            'hit_ratio_TC': hit_ratio_BH,
            'annual_return': annual_return_BH,
            'annual_return_TC': annual_return_BH,
            'annual_volatility': annual_volatility_BH,
            'annual_volatility_TC': annual_volatility_BH
        }, index=[0])
        
        evaluation_df = pd.concat([evaluation_df, new_row_bh], ignore_index=True)
    
    return dictionary, evaluation_df

In [30]:
import pandas as pd
import numpy as np

def calculate_strategy_returns(df, transaction_cost=0.006):
    """
    Calculate the relative return of an investment strategy with and without transaction costs.
    
    Parameters:
    df (pd.DataFrame): The dataframe containing at least the 'Price' column, 'predicted_label' column, and datetime index.
    transaction_cost (float): The transaction cost per trade as a fraction.
    
    Returns:
    pd.DataFrame: The dataframe with additional columns 'Strategy_Return' and 'Strategy_Return_TC' that contain the calculated strategy returns without and with transaction costs.
    """
    # Ensure the dataframe is sorted by the index
    df = df.sort_index()

    # Calculate the percentage change in price
    df['Pct_Change'] = df['Price'].pct_change().fillna(0)

    # Calculate the strategy returns without transaction costs
    df['Strategy_Return'] = df['Pct_Change'] * df['predicted label']

    # Apply transaction costs only when there's a trade (label is -1 or 1)
    df['Transaction_Cost'] = df['predicted label'].abs() * transaction_cost

    # Calculate the strategy returns with transaction costs
    df['Strategy_Return_TC'] = df['Strategy_Return'] - df['Transaction_Cost']

    # For no trade, the return should be 0
    df.loc[df['predicted label'] == 0, ['Strategy_Return', 'Strategy_Return_TC']] = 0

    return df

def evaluate_strategy_financially(dictionary, periods, transaction_cost=0.006):
    """
    Evaluate the financial performance of various trading strategies and compare them to a Buy and Hold strategy.
    
    Parameters:
    dictionary (dict): Dictionary containing the trading strategies and their test data.
    periods (str): 'daily' or 'weekly' indicating the frequency of the data points.
    transaction_cost (float): The transaction cost per trade as a fraction.
    
    Returns:
    tuple: Updated dictionary with test results and a DataFrame containing the evaluation metrics.
    """
    columns = [
        'labeling_method', 
        'currency', 
        'overall_return', 'overall_return_TC',
        'sharpe_ratio', 'sharpe_ratio_TC',
        'maximum_drawdown', 'maximum_drawdown_TC',
        'mean_return', 'mean_return_TC',
        'hit_ratio', 'hit_ratio_TC',
        'annual_return', 'annual_return_TC',
        'annual_volatility', 'annual_volatility_TC'
    ]
    
    evaluation_df = pd.DataFrame(columns=columns)
    
    if periods not in ["daily", "weekly"]:
        raise ValueError("Invalid value for periods. Use 'daily' or 'weekly'.")

    for labeling_method in dictionary:
        for currency in dictionary[labeling_method]:
            # Calculate strategy returns with and without transaction costs
            strategy_df = dictionary[labeling_method][currency]["test"]
            strategy_df = calculate_strategy_returns(strategy_df, transaction_cost)
            dictionary[labeling_method][currency]["test"] = strategy_df
            
            # Metrics without transaction costs
            overall_return = calculate_overall_strategy_return(strategy_df, 'Strategy_Return')
            sharpe_ratio = calculate_sharpe_ratio(strategy_df, 'Strategy_Return')
            max_drawdown = calculate_max_drawdown(strategy_df, 'Strategy_Return')
            mean_return = calculate_mean_return(strategy_df, 'Strategy_Return')
            hit_ratio = calculate_hit_ratio(strategy_df, 'Strategy_Return')
            annual_return = calculate_annual_return(strategy_df, periods, 'Strategy_Return')
            annual_volatility = calculate_annual_volatility(strategy_df, periods, 'Strategy_Return')
            
            # Metrics with transaction costs
            overall_return_TC = calculate_overall_strategy_return(strategy_df, 'Strategy_Return_TC')
            sharpe_ratio_TC = calculate_sharpe_ratio(strategy_df, 'Strategy_Return_TC')
            max_drawdown_TC = calculate_max_drawdown(strategy_df, 'Strategy_Return_TC')
            mean_return_TC = calculate_mean_return(strategy_df, 'Strategy_Return_TC')
            hit_ratio_TC = calculate_hit_ratio(strategy_df, 'Strategy_Return_TC')
            annual_return_TC = calculate_annual_return(strategy_df, periods, 'Strategy_Return_TC')
            annual_volatility_TC = calculate_annual_volatility(strategy_df, periods, 'Strategy_Return_TC')
            
            new_row = pd.DataFrame({
                'labeling_method': labeling_method, 
                'currency': currency, 
                'overall_return': overall_return,
                'overall_return_TC': overall_return_TC,
                'sharpe_ratio': sharpe_ratio,
                'sharpe_ratio_TC': sharpe_ratio_TC,
                'maximum_drawdown': max_drawdown,
                'maximum_drawdown_TC': max_drawdown_TC,
                'mean_return': mean_return,
                'mean_return_TC': mean_return_TC,
                'hit_ratio': hit_ratio,
                'hit_ratio_TC': hit_ratio_TC,
                'annual_return': annual_return,
                'annual_return_TC': annual_return_TC,
                'annual_volatility': annual_volatility,
                'annual_volatility_TC': annual_volatility_TC
            }, index=[0])
            
            evaluation_df = pd.concat([evaluation_df, new_row], ignore_index=True)
    
    # Calculate metrics for Buy and Hold strategy for each currency, with transaction costs
    for currency in dictionary[labeling_method]:
        strategy_df = dictionary[labeling_method][currency]["test"]
        
        # Calculate buy-and-hold metrics based on the price percentage change
        strategy_df['Pct_Change'] = strategy_df['Price'].pct_change().fillna(0)
        returns_bh = strategy_df['Pct_Change']
        returns_bh_tc = returns_bh - transaction_cost  # Apply daily transaction costs
        
        overall_return_BH = (returns_bh + 1).prod() - 1
        overall_return_BH_TC = (returns_bh_tc + 1).prod() - 1
        
        sharpe_ratio_BH = calculate_sharpe_ratio(strategy_df, 'Pct_Change')
        sharpe_ratio_BH_TC = calculate_sharpe_ratio(pd.DataFrame(returns_bh_tc, columns=['Pct_Change']), 'Pct_Change')
        
        max_drawdown_BH = calculate_max_drawdown(strategy_df, 'Pct_Change')
        max_drawdown_BH_TC = calculate_max_drawdown(pd.DataFrame(returns_bh_tc, columns=['Pct_Change']), 'Pct_Change')
        
        mean_return_BH = calculate_mean_return(strategy_df, 'Pct_Change')
        mean_return_BH_TC = calculate_mean_return(pd.DataFrame(returns_bh_tc, columns=['Pct_Change']), 'Pct_Change')
        
        hit_ratio_BH = calculate_hit_ratio(strategy_df, 'Pct_Change')
        hit_ratio_BH_TC = calculate_hit_ratio(pd.DataFrame(returns_bh_tc, columns=['Pct_Change']), 'Pct_Change')
        
        annual_return_BH = calculate_annual_return(strategy_df, periods, 'Pct_Change')
        annual_return_BH_TC = calculate_annual_return(pd.DataFrame(returns_bh_tc, columns=['Pct_Change']), periods, 'Pct_Change')
        
        annual_volatility_BH = calculate_annual_volatility(strategy_df, periods, 'Pct_Change')
        annual_volatility_BH_TC = calculate_annual_volatility(pd.DataFrame(returns_bh_tc, columns=['Pct_Change']), periods, 'Pct_Change')
        
        # Append the Buy and Hold results to the evaluation dataframe
        new_row_bh = pd.DataFrame({
            'labeling_method': 'Buy and Hold', 
            'currency': currency, 
            'overall_return': overall_return_BH,
            'overall_return_TC': overall_return_BH_TC,
            'sharpe_ratio': sharpe_ratio_BH,
            'sharpe_ratio_TC': sharpe_ratio_BH_TC,
            'maximum_drawdown': max_drawdown_BH,
            'maximum_drawdown_TC': max_drawdown_BH_TC,
            'mean_return': mean_return_BH,
            'mean_return_TC': mean_return_BH_TC,
            'hit_ratio': hit_ratio_BH,
            'hit_ratio_TC': hit_ratio_BH_TC,
            'annual_return': annual_return_BH,
            'annual_return_TC': annual_return_BH_TC,
            'annual_volatility': annual_volatility_BH,
            'annual_volatility_TC': annual_volatility_BH_TC
        }, index=[0])
        
        evaluation_df = pd.concat([evaluation_df, new_row_bh], ignore_index=True)
    
    return dictionary, evaluation_df

# Please ensure all utility functions are defined elsewhere as per the original setup.
